In [2]:
# 🔌 Cargar la extensión de Kedro en Jupyter
%load_ext kedro.ipython
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [4]:
df = catalog.load("transactions_model_ready")   # lee el dataset del catálogo
list(df.columns)  

[10/18/25 22:36:46] INFO     Loading data from transactions_model_ready (ParquetDataset)...    ]8;id=973178;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=596237;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\


[
    'TransactionID',
    'CustomerID',
    'CustLocation',
    'TransactionDate',
    'TransactionTime',
    'TransactionAmount (INR)',
    'hour',
    'day_of_week',
    'is_weekend',
    'is_night',
    'time_since_last_transaction',
    'is_rapid_transaction',
    'mu',
    'sigma',
    'amount_zscore',
    'is_anomalous_amount',
    'is_high_value',
    'hour_sin',
    'hour_cos',
    'day_sin',
    'day_cos'
]